In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))
from functions import *
from lammps_data_reader import *

# Create cif

In [14]:
df = read_pdb('../files/POSCAR.pdb')
h2o = read_pdb('../files/h2o.pdb')
h2o.loc[:, list('xyz')] -= h2o.loc[2, ['x', 'y', 'z']].values
h2o.loc[:, list('xyz')] += np.array([5, 5, 5])

parameters, _ = read_cif_data('../files/NU1000h2o_1_1_1.cif')
df.loc[:, list('xyz')] = cartesian_to_lattice(df[['x', 'y', 'z']].values, parameters)
h2o.loc[:, list('xyz')] = cartesian_to_lattice(h2o[['x', 'y', 'z']].values, parameters)

write_cif('../files/h2o.cif', parameters, h2o)
write_cif('../files/POSCAR.cif', parameters, df)

In [16]:
!julia run_packmol.jl

The latest version of Julia in the `release` channel is 1.10.4+0.x64.w64.mingw32. You currently have `1.9.3+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.4+0.x64.w64.mingw32 and update the `release` channel to that version.
ERROR: SystemError: opening file "c:\\Users\\lt0828\\Desktop\\PhD\\ru-project\\notebooks\\run_packmol.jl": No such file or directory
Stacktrace:
  [1] systemerror(p::String, errno::Int32; extrainfo::Nothing)
    @ Base .\error.jl:176
  [2] #systemerror#82
    @ .\error.jl:175 [inlined]
  [3] systemerror
    @ .\error.jl:175 [inlined]
  [4] open(fname::String; lock::Bool, read::Nothing, write::Nothing, create::Nothing, truncate::Nothing, append::Nothing)
    @ Base .\iostream.jl:293
  [5] open
    @ .\iostream.jl:275 [inlined]
  [6] open(f::Base.var"#418#419"{String}, args::String; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ Base .\io.jl:393
  [7] open
    @ .\io.jl:392 [inlined]
  

# Create data

## Packmol

In [26]:
df = read_pdb('../files/POSCAR.pdb')
r = (np.abs(df.loc[:, list('xyz')].min() - df.loc[:, list('xyz')].max())/2).mean()
v = 4/3 * np.pi * r**3

# Задаем объем в ангстремах
v_angstrom = v = 15 ** 3 - v

# Константы
density = 1  # г/см^3 (плотность воды)
molecular_mass = 18  # г/моль (молекулярная масса воды)
avogadro_number = 6.022e23  # Число Авогадро

# Переводим объем в см^3 (1 см^3 = 10^24 Å^3)
v_cm3 = v_angstrom / 1e24

# Находим количество молекул воды
# Масса воды в данном объеме (граммы) = плотность * объем
mass_water = density * v_cm3

# Количество молекул воды = (масса воды / масса одной молекулы воды)
mass_per_molecule = molecular_mass / avogadro_number
num_molecules = mass_water / mass_per_molecule

num_molecules


110.44374079629975

## Lammps

In [15]:
data = LammpsData('../files/data.NU1000h2o_ru')
data.read()
data.atoms

,id,mol_id,atom_type,q,x,y,z
0,1,444,1,0.0,19.94738,2.47283,0.00068
1,2,444,1,0.0,1.84776,34.52815,14.78947
2,3,444,1,0.0,21.65848,0.00790,1.85098
3,4,444,1,0.0,-1.85260,34.52519,1.70710
4,5,444,1,0.0,0.00261,32.05369,0.00233
...,...,...,...,...,...,...,...
1139,1140,444,5,0.0,10.39751,13.99661,30.55229
1140,1141,444,5,0.0,13.03755,18.57078,30.58569
1141,1142,444,6,0.0,11.89645,20.68152,31.77468
1142,1143,444,4,0.0,9.79971,19.73986,16.55413
